In [11]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType,StringType,IntegerType,FloatType
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import os
import matplotlib.pyplot as plt
from pyspark.sql.functions import count,col,udf,year,month,avg,round,weekofyear,isnull,dayofweek

In [5]:
spark = SparkSession.builder  \
        .appName("ANALYST_TASK") \
          .getOrCreate()

In [28]:
medications=spark.read.format('csv').option("delimiter",",").option("header","true").load("medications.csv")
allergies=spark.read.csv('allergies.csv',header=True)
encounters=spark.read.csv('encounters.csv',header=True)
patients=spark.read.csv('patients.csv',header=True)
procedures=spark.read.csv('procedures.csv',header=True)

In [31]:
medications.first()

Row(START='1997-09-20', STOP='NA', PATIENT='2043c57c-1085-45a3-87e8-a223d5b6693d', ENCOUNTER='3710b40a-bcc0-4c0f-a204-e47a5f3aec98', CODE='316049', DESCRIPTION='Hydromorphone 325 MG', COST='214.96', DISPENSES='265', TOTALCOST='56964.4', REASONCODE='59621000', REASONDESCRIPTION='Pain Relief')

In [32]:
allergies.first()

Row(START='1983-07-12', STOP='NA', PATIENT='7341aae3-7606-456e-b3c1-f67d1bc6b19b', ENCOUNTER='8ea228c8-f853-431e-969b-c7085602d2c1', CODE='300913006', DESCRIPTION='Shellfish allergy')

In [33]:
encounters.first()

Row(Id='2590963f-222b-4870-b8c7-86d07f4d44b3', START='1959-06-18 09:43:49', STOP='1959-06-25 21:37:31', PATIENT='3d8e57b2-3de5-4fb6-95b4-8370270c5cc2', PROVIDER='fa6006c2-c562-3278-859b-96b266fc3ea1', ENCOUNTERCLASS='ambulatory', CODE='424441002', DESCRIPTION='Prenatal initial visit', COST='105.37', REASONCODE='72892002', REASONDESCRIPTION='Normal pregnancy')

In [34]:
patients.first()

Row(Id='3d8e57b2-3de5-4fb6-95b4-8370270c5cc2', BIRTHDATE='1943-03-11', DEATHDATE='NA', SSN='999-86-7250', DRIVERS='S99939389', PASSPORT='X3970685X', PREFIX='Mrs.', FIRST='Allyn942', LAST='Kreiger457', SUFFIX='NA', MAIDEN='Bartoletti50', MARITAL='M', RACE='asian', ETHNICITY='asian_indian', GENDER='F', BIRTHPLACE='Muhlenberg  Pennsylvania  US', ADDRESS='372 Marks Heights Unit 20', CITY='Middle Paxton', STATE='Pennsylvania', ZIP='NA')

In [35]:
procedures.first()

Row(DATE='2009-09-17', PATIENT.x='553b00b2-347c-48ec-90ee-f59924ff50eb', ENCOUNTER='d08babad-74ed-4490-8830-24af7310cac7', CODE.x='76601001', DESCRIPTION.x='Intramuscular injection', COST.x='1858.49', REASONCODE.x='NA', REASONDESCRIPTION.x='NA')

In [36]:
encounters.printSchema()

root
 |-- Id: string (nullable = true)
 |-- START: string (nullable = true)
 |-- STOP: string (nullable = true)
 |-- PATIENT: string (nullable = true)
 |-- PROVIDER: string (nullable = true)
 |-- ENCOUNTERCLASS: string (nullable = true)
 |-- CODE: string (nullable = true)
 |-- DESCRIPTION: string (nullable = true)
 |-- COST: string (nullable = true)
 |-- REASONCODE: string (nullable = true)
 |-- REASONDESCRIPTION: string (nullable = true)



### Rename columns name in a dataframe

In [42]:
encounters=encounters.withColumnRenamed('Id','ENCOUNTER_ID')
medications=medications.withColumnRenamed('START','START_MEDICATIONS')
medications=medications.withColumnRenamed('STOP','STOP_MEDICATIONS')
medications=medications.withColumnRenamed('START','START_MEDICATIONS')
medications=medications.withColumnRenamed('DECRIPTION','MEDICAL_DESCRIPTION')

+--------------------+----------+---------+------------+------------+
|                  Id| BIRTHDATE|DEATHDATE|      MAIDEN|  MAIDEN_NEW|
+--------------------+----------+---------+------------+------------+
|3d8e57b2-3de5-4fb...|1943-03-11|       NA|Bartoletti50|BARTOLETTI50|
|7f4ea9fb-f436-411...|1980-09-28|       NA|          NA|          NA|
|553b00b2-347c-48e...|1973-02-22|       NA|  Labadie908|  LABADIE908|
|50f799aa-740c-4da...|1955-03-30|       NA|          NA|          NA|
+--------------------+----------+---------+------------+------------+
only showing top 4 rows

